In [ ]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import torch
import cv2
import numpy as np
from ultralytics import YOLO

# Загрузка модели YOLOv8
model = YOLO('yolov8n.pt')  


def detect_objects(image_path):
    
    results = model(image_path)

    
    detections = results[0].boxes.xyxy.cpu().numpy() 
    class_ids = results[0].boxes.cls.cpu().numpy()  
    confidences = results[0].boxes.conf.cpu().numpy()  

    return detections, class_ids, confidences


def load_image():
    global panelA, panelB
    path = filedialog.askopenfilename()
    if len(path) > 0:
        image = cv2.imread(path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image_pil = Image.fromarray(image_rgb)
        image_tk = ImageTk.PhotoImage(image_pil)

        detections, class_ids, confidences = detect_objects(path)
        for box, class_id, confidence in zip(detections, class_ids, confidences):
            xmin, ymin, xmax, ymax = map(int, box)
            label = model.names[int(class_id)]
            cv2.rectangle(image_rgb, (xmin, ymin), (xmax, ymax), (255, 0, 0), 2)
            cv2.putText(image_rgb, f'{label} {confidence:.2f}', (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        image_pil = Image.fromarray(image_rgb)
        image_tk = ImageTk.PhotoImage(image_pil)

        if panelA is None or panelB is None:
            panelA = tk.Label(image=image_tk)
            panelA.image = image_tk
            panelA.pack(side="left", padx=10, pady=10)

            panelB = tk.Label(text="Распознано объектов: {}".format(len(detections)))
            panelB.pack(side="right", padx=10, pady=10)
        else:
            panelA.configure(image=image_tk)
            panelA.image = image_tk

            panelB.configure(text="Распознано объектов: {}".format(len(detections)))


root = tk.Tk()
panelA = None
panelB = None

btn = tk.Button(root, text="Загрузить изображение", command=load_image)
btn.pack(side="bottom", fill="both", expand="yes", padx=10, pady=10)

root.mainloop()